In [ ]:
import torch.nn as nn
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset,DataLoader
from datasets import load_dataset

In [ ]:
class my_dataset(Dataset):
    def __init__(self,split):
        self.dataset=load_dataset("seamew/Weibo",split=split)
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self,idx):
        text=self.dataset[idx]["text"]
        label=self.dataset[idx]["label"]
        return text,label

In [ ]:
train_dataset=my_dataset("train")

In [ ]:
all_label=[]
for i in range(len(train_dataset)):
    if train_dataset[i][1] not in all_label:
        all_label.append(train_dataset[i][1])
len(all_label)

In [ ]:
from transformers import BertTokenizer
token=BertTokenizer.from_pretrained("bert-base-chinese")

In [ ]:
def collate_fn(data):
    sents=[i[0] for i in data]
    labels=[i[1] for i in data]
    data = token.batch_encode_plus(
        batch_text_or_text_pairs=sents,
        truncation=True,
        padding="max_length",
        max_length=32,
        return_tensors='pt',
        return_length=True)
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    return input_ids, attention_mask, token_type_ids, labels
loader=DataLoader(dataset=train_dataset,batch_size=16,shuffle=True,collate_fn=collate_fn)

In [ ]:
from transformers import BertModel
pre_model=BertModel.from_pretrained("bert-base-chinese")
#for param in pre_model.parameters():
#    param.requires_grad_(False)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break
out=pre_model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
out.last_hidden_state.shape

In [ ]:
#定义分类任务
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc=nn.Linear(768,8)
    def forward(self,input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pre_model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
            #只取第一个，也就是CLS进行预测
        out=self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out
model=Model()
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

In [ ]:
from transformers import AdamW
optim=AdamW(model.parameters(),lr=1e-4)
criterion=nn.CrossEntropyLoss()
model.train()
sched = torch.optim.lr_scheduler.StepLR(optim, step_size=1, gamma=0.5)
for epoch in range(2):
    for i,(input_ids, attention_mask, token_type_ids,labels) in enumerate(loader):
        out=model(input_ids, attention_mask, token_type_ids)
        loss=criterion(out,labels)
        loss.backward()
        optim.step()
        optim.zero_grad()
        if i % 5 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(i, loss.item(), accuracy)
    sched.step()